In [1]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot  as plt
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin1")

In [3]:
df.columns= ['target', 'ids', 'date', 'queryFlag', 'user', 'text']

In [4]:
x = df.drop(['ids', 'date', 'queryFlag', 'user'], axis=1)
X_train, X_test=train_test_split(x)

In [5]:
X_train

,target,text
705923,0,@ValeriaLuciana i miss you stup*idd! i need y...
146128,0,@MatthewCGoto so sad I won't be joining have ...
1342697,4,Listening to iTunes on shuffle and straighteni...
57078,0,I have dark spots on my eyes... I thought a be...
1284440,4,"@ausher8 I guess you were an exception, to hav..."
...,...,...
432433,0,Not wanting to go to work
1421683,4,no rain! and the wedding was beautiful. the b...
1540670,4,i'll be back when Spain starts playing against...
81101,0,@IamBubbles We don't have Dunkin's here.


In [6]:
X_train=X_train.sort_values('target')

In [7]:
y_train=X_train['target']
X_train = X_train.drop('target', axis=1)

In [8]:
y_train

705923     0
461519     0
681877     0
33261      0
482294     0
          ..
1480714    4
1532748    4
1464726    4
1305263    4
907167     4
Name: target, Length: 1199999, dtype: int64

In [19]:
len(y_train[y_train==4])

599890

In [10]:
X_test = X_test.sort_values('target')
y_test = X_test['target']
X_test = X_test.drop('target', axis=1)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cvz = CountVectorizer(stop_words='english', max_features=10000)

docMatrix=cvz.fit_transform(X_train.text)

In [18]:
docMatrix

<1199999x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 6952618 stored elements in Compressed Sparse Row format>

In [14]:
cvz.get_feature_names_out()[500:600]

array(['amp', 'amsterdam', 'amuse', 'amused', 'amusing', 'amy', 'ana',
       'anal', 'analysis', 'anatomy', 'anberlin', 'ancient', 'anderson',
       'andre', 'andrea', 'andrew', 'android', 'andy', 'andyclemmensen',
       'andyhurleyday', 'ang', 'angel', 'angela', 'angeles', 'angels',
       'anger', 'angie', 'angry', 'anh', 'animal', 'animals', 'animated',
       'animation', 'anime', 'ankle', 'ankles', 'ann', 'anna', 'anne',
       'annie', 'anniversary', 'announce', 'announced', 'announcement',
       'annoy', 'annoyed', 'annoying', 'annoys', 'annual', 'ano', 'anoop',
       'anoopdoggdesai', 'ans', 'answer', 'answered', 'answering',
       'answers', 'ant', 'anthem', 'anthony', 'anti', 'antibiotics',
       'anticipation', 'antonio', 'ants', 'anxiety', 'anxious',
       'anxiously', 'any1', 'anybody', 'anymore', 'anythin', 'anytime',
       'anyways', 'anywho', 'ap', 'apa', 'apart', 'apartment', 'api',
       'aplusk', 'apologies', 'apologize', 'app', 'apparently',
       'appeal

In [20]:
neg_batches = np.linspace(0,600109,10).astype(int)
i=0
neg_tf = []
while i < len(neg_batches)-1:
    batch_result = np.sum(docMatrix[neg_batches[i]:neg_batches[i+1]].toarray(),axis=0)
    neg_tf.append(batch_result)
    print (neg_batches[i+1],"entries' term freuquency calculated")
    i += 1

66678 entries' term freuquency calculated
133357 entries' term freuquency calculated
200036 entries' term freuquency calculated
266715 entries' term freuquency calculated
333393 entries' term freuquency calculated
400072 entries' term freuquency calculated
466751 entries' term freuquency calculated
533430 entries' term freuquency calculated
600109 entries' term freuquency calculated


In [21]:
pos_batches = np.linspace(600109,1199999,10).astype(int)
i=0
pos_tf = []
while i < len(pos_batches)-1:
    batch_result = np.sum(docMatrix[pos_batches[i]:pos_batches[i+1]].toarray(),axis=0)
    pos_tf.append(batch_result)
    print (pos_batches[i+1],"entries' term freuquency calculated")
    i += 1

666763 entries' term freuquency calculated
733417 entries' term freuquency calculated
800072 entries' term freuquency calculated
866726 entries' term freuquency calculated
933381 entries' term freuquency calculated
1000035 entries' term freuquency calculated
1066690 entries' term freuquency calculated
1133344 entries' term freuquency calculated
1199999 entries' term freuquency calculated


In [148]:
neg = np.sum(neg_tf,axis=0)
pos = np.sum(pos_tf,axis=0)
term_freq_df2= pd.DataFrame([neg,pos], columns=cvz.get_feature_names()).transpose()
term_freq_df2.columns = ['negative', 'positive']
term_freq_df2['total'] = term_freq_df2['negative'] + term_freq_df2['positive']
term_freq_df2.sort_values(by='total', ascending=False).iloc[:10]

,negative,positive,total
just,48080,47215,95295
good,21797,46602,68399
day,29793,35353,65146
like,30782,28215,58997
quot,19639,34411,54050
http,18294,35447,53741
today,28531,22644,51175
work,34073,14624,48697
going,25247,23232,48479
love,12586,35764,48350


In [164]:
term_freq_df2['pos_rate'] = term_freq_df2['positive']/( term_freq_df2['positive']+ term_freq_df2['negative'])
term_freq_df2['pos_freq'] = term_freq_df2['positive']/( term_freq_df2['positive'].sum())
term_freq_df2.sort_values(by='total', ascending=False).iloc[:10]

,negative,positive,total,pos_rate,pos_freq
just,48080,47215,95295,0.495461,0.013372
good,21797,46602,68399,0.681326,0.013199
day,29793,35353,65146,0.542673,0.010013
like,30782,28215,58997,0.478245,0.007991
quot,19639,34411,54050,0.636651,0.009746
http,18294,35447,53741,0.659590,0.010039
today,28531,22644,51175,0.442482,0.006413
work,34073,14624,48697,0.300306,0.004142
going,25247,23232,48479,0.479218,0.006580
love,12586,35764,48350,0.739690,0.010129


In [165]:
from scipy.stats import norm, hmean
def normcdf(x):
    return norm.cdf(x, x.mean(), x.std())
term_freq_df2['pos_rate_cdf'] = normcdf(term_freq_df2['pos_rate'])
term_freq_df2['pos_freq_cdf'] = normcdf(term_freq_df2['pos_freq'])
term_freq_df2['pos_cdf_hmean'] = hmean([term_freq_df2['pos_rate_cdf'], term_freq_df2['pos_freq_cdf']])
term_freq_df2.sort_values(by='pos_cdf_hmean',ascending=False).iloc[:10]

,negative,positive,total,pos_rate,pos_freq,pos_rate_cdf,pos_freq_cdf,pos_cdf_hmean
welcome,464,5026,5490,0.915483,0.001423,0.994879,0.998408,0.996641
thank,1724,11840,13564,0.872899,0.003353,0.989342,1.000000,0.994642
thanks,4331,25920,30251,0.856831,0.007341,0.986176,1.000000,0.993040
www,1097,5733,6830,0.839385,0.001624,0.981852,0.999658,0.990675
awesome,2827,10766,13593,0.792025,0.003049,0.963959,1.000000,0.981649
follow,1917,6900,8817,0.782579,0.001954,0.959050,0.999982,0.979088
glad,1714,6116,7830,0.781098,0.001732,0.958233,0.999862,0.978605
enjoy,1212,4455,5667,0.786130,0.001262,0.960955,0.995188,0.977772
yay,2368,7928,10296,0.770008,0.002245,0.951689,0.999999,0.975246
amazing,2069,6792,8861,0.766505,0.001924,0.949460,0.999976,0.974064


In [166]:
term_freq_df2['neg_rate'] = term_freq_df2['negative']/( term_freq_df2['positive']+ term_freq_df2['negative'])
term_freq_df2['neg_freq'] = term_freq_df2['negative']/( term_freq_df2['negative'].sum())
term_freq_df2.sort_values(by='total', ascending=False).iloc[:10]

,negative,positive,total,pos_rate,pos_freq,pos_rate_cdf,pos_freq_cdf,pos_cdf_hmean,neg_rate,neg_freq
just,48080,47215,95295,0.495461,0.013372,0.480623,1.0,0.649218,0.504539,0.013274
good,21797,46602,68399,0.681326,0.013199,0.866305,1.0,0.928364,0.318674,0.006018
day,29793,35353,65146,0.542673,0.010013,0.596956,1.0,0.747618,0.457327,0.008225
like,30782,28215,58997,0.478245,0.007991,0.438085,1.0,0.609262,0.521755,0.008498
quot,19639,34411,54050,0.636651,0.009746,0.796965,1.0,0.887012,0.363349,0.005422
http,18294,35447,53741,0.659590,0.010039,0.834898,1.0,0.910021,0.340410,0.005051
today,28531,22644,51175,0.442482,0.006413,0.352499,1.0,0.521256,0.557518,0.007877
work,34073,14624,48697,0.300306,0.004142,0.103090,1.0,0.186911,0.699694,0.009407
going,25247,23232,48479,0.479218,0.006580,0.440475,1.0,0.611569,0.520782,0.006970
love,12586,35764,48350,0.739690,0.010129,0.929573,1.0,0.963501,0.260310,0.003475


In [167]:
term_freq_df2['neg_rate_cdf'] = normcdf(term_freq_df2['neg_rate'])
term_freq_df2['neg_freq_cdf'] = normcdf(term_freq_df2['neg_freq'])
term_freq_df2['neg_cdf_hmean'] = hmean([term_freq_df2['neg_rate_cdf'], term_freq_df2['neg_freq_cdf']])
term_freq_df2.sort_values(by='neg_cdf_hmean',ascending=False).iloc[:10]

,negative,positive,total,pos_rate,pos_freq,pos_rate_cdf,pos_freq_cdf,pos_cdf_hmean,neg_rate,neg_freq,neg_rate_cdf,neg_freq_cdf,neg_cdf_hmean
sad,20879,1095,21974,0.049832,0.000310,0.002370,0.680208,0.004723,0.950168,0.005764,0.997630,1.000000,0.998814
hurts,5419,359,5778,0.062132,0.000102,0.003001,0.501491,0.005967,0.937868,0.001496,0.996999,0.999595,0.998295
sick,10891,1058,11949,0.088543,0.000300,0.004895,0.671814,0.009720,0.911457,0.003007,0.995105,1.000000,0.997546
sucks,7411,736,8147,0.090340,0.000208,0.005056,0.595490,0.010028,0.909660,0.002046,0.994944,0.999998,0.997465
poor,5605,562,6167,0.091130,0.000159,0.005129,0.552454,0.010163,0.908870,0.001547,0.994871,0.999742,0.997301
ugh,6801,753,7554,0.099682,0.000213,0.005973,0.599642,0.011828,0.900318,0.001878,0.994027,0.999990,0.997000
missing,5488,746,6234,0.119666,0.000211,0.008441,0.597933,0.016646,0.880334,0.001515,0.991559,0.999657,0.995591
hate,12908,1987,14895,0.133400,0.000563,0.010619,0.848797,0.020975,0.866600,0.003564,0.989381,1.000000,0.994662
headache,4020,314,4334,0.072450,0.000089,0.003644,0.490161,0.007235,0.927550,0.001110,0.996356,0.992294,0.994321
miss,22977,4255,27232,0.156250,0.001205,0.015332,0.993107,0.030197,0.843750,0.006344,0.984668,1.000000,0.992275


In [229]:
X_Tran = cvz.transform(X_test.text)

In [230]:
X_Tran

<400000x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 2313917 stored elements in Compressed Sparse Row format>

In [103]:
cvz.get_feature_names()[7787]

'shop'

In [209]:
data= term_freq_df2.filter(items=[cvz.get_feature_names_out()[index] for  i, index in enumerate(X.nonzero()[1])], axis=0)


In [210]:
type(data)

pandas.core.frame.DataFrame

In [378]:
def evaluation(data):
    if np.sum(data['pos_cdf_hmean']) > np.sum(data['neg_cdf_hmean']):
        return 4
    # elif np.sum(data['pos_cdf_hmean']) > np.sum(data['neg_cdf_hmean']):
    #     return 0
    else: 
        return 0

In [366]:
y_test.iloc[392000]

4

In [368]:
print(evaluation(term_freq_df2.filter(items=[cvz.get_feature_names_out()[index] for  i, index in enumerate(X_Tran[312500].nonzero()[1])], axis=0)))
 

4


In [360]:
np.sum(data['neg_cdf_hmean'])

2.5435742918114657

In [322]:
X_test.text.iloc[392000]

'@chipcoffey I hope you get invited back to Paranormal State..I think you are a great part to the show '

In [328]:
term_freq_df2.filter(items=[cvz.get_feature_names_out()[index] for  i, index in enumerate(X_Tran[392000].nonzero()[1])], axis=0)


,negative,positive,total,pos_rate,pos_freq,pos_rate_cdf,pos_freq_cdf,pos_cdf_hmean,neg_rate,neg_freq,neg_rate_cdf,neg_freq_cdf,neg_cdf_hmean
great,6168,18949,25117,0.754429,0.005367,0.941141,1.000000,0.969678,0.245571,0.001703,0.058859,0.999940,0.111173
hope,11031,14497,25528,0.567886,0.004106,0.656349,1.000000,0.792525,0.432114,0.003045,0.343651,1.000000,0.511519
invited,356,202,558,0.362007,0.000057,0.189476,0.462015,0.268740,0.637993,0.000098,0.810524,0.498360,0.617217
state,568,508,1076,0.472119,0.000144,0.423095,0.538947,0.474046,0.527881,0.000157,0.576905,0.554206,0.565328
think,16519,14573,31092,0.468706,0.004127,0.414790,1.000000,0.586362,0.531294,0.004561,0.585210,1.000000,0.738338


In [289]:
len(X_test)

400000

In [369]:
X_Tran[390000:392589]

<2589x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 14747 stored elements in Compressed Sparse Row format>

In [379]:
y_pred=[]
for k, X_ele in enumerate(X_Tran[390000:392589]):
    y_pred.append(evaluation(term_freq_df2.filter(items=[cvz.get_feature_names_out()[index] for  i, index in enumerate(X_ele.nonzero()[1])], axis=0)))


In [343]:
len(y_pred)

2589

In [251]:
from sklearn.metrics import accuracy_score

In [380]:
sco = accuracy_score(y_test[390000:392589], y_pred)

In [381]:
sco

0.7211278485901893